In [1]:
import networkx as nx
import requests
import matplotlib as plt

In [2]:
url = 'https://www.ia.pw.edu.pl/~mkamola/dataset-big/4.txt'
r = requests.get(url, allow_redirects=True)
open('4.txt', 'wb').write(r.content)

185860

Zbadaj jaki jest rząd i rozmiar całej sieci: pierwotnej oraz po usunięciu pętli i duplikatów krawędzi (1);

In [3]:

G_multi = nx.read_edgelist("4.txt", create_using=nx.MultiGraph)
#G = nx.read_edgelist("4.txt", create_using=nx.Graph)
#G_di = nx.read_edgelist("4.txt", create_using=nx.DiGraph)
#G_multidi = nx.read_edgelist("4.txt", create_using=nx.MultiDiGraph)

In [4]:
I1 = nx.info(G_multi) 
#I2 = nx.info(G) 
#I3 =nx.info(G_di) 
#I4 =nx.info(G_multidi) 
print(I1 + '\n')
#print(I2 + '\n')
#print(I3)
#print(I4)

Name: 
Type: MultiGraph
Number of nodes: 6988
Number of edges: 5537
Average degree:   1.5847



In [5]:
nx.number_of_selfloops(G_multi)

256

In [6]:
#list(G_multi.nodes)

In [7]:
#list(G_multi.edges)

In [8]:
#list(G_multi.degree)

In [9]:
G_multi=nx.Graph(G_multi)

In [10]:
#nx.draw(G_multi, with_labels=True, font_weight='bold')

In [11]:
G_multi.remove_edges_from(nx.selfloop_edges(G_multi))

In [12]:
nx.number_of_selfloops(G_multi)

0

In [13]:
print(I1 + '\n')

Name: 
Type: MultiGraph
Number of nodes: 6988
Number of edges: 5537
Average degree:   1.5847



Wyodrębnij największą składową spójną, zbadaj jej rząd i rozmiar (1);

In [14]:
Set = [len(c) for c in sorted(nx.connected_components(G_multi), key=len, reverse=True)]

In [15]:
largest_cc = max(nx.connected_components(G_multi), key=len)

In [16]:
G_biggest=G_multi.subgraph(largest_cc)

In [17]:
Inf = nx.info(G_biggest)
print(Inf)

Name: 
Type: Graph
Number of nodes: 626
Number of edges: 742
Average degree:   2.3706


In [18]:
nx.number_of_nodes(G_biggest)
nx.number_of_edges(G_biggest)

742

Wyznacz aproksymacje średniej długości ścieżki, operując na próbie losowej 100, 1000 i 10 tys. par wierzchołków (2);

In [19]:
import random


In [25]:
n_set = [100, 1000, 10000]
for i in n_set:
    random_graph = random.sample(G_multi.nodes, i)
    create_graph = G_multi.subgraph(random_graph)
    print(nx.info(create_graph))
    try:
        aspl = nx.average_shortest_path_length(create_graph)
        print(i,aspl)
    except nx.NetworkXError:
        print('Cannot count length')
nx.average_shortest_path_length(G_multi)

Name: 
Type: Graph
Number of nodes: 0
Number of edges: 0



NetworkXPointlessConcept: the null graph has no paths, thus there is no averageshortest path length

In [ ]:
def top_degree(graph, n):
 degree = nx.degree(graph)
 sdegree = sorted(degree.iteritems(),
key=lambda n: -n[1])
 return sdegree[0:n]


In [ ]:
top_degree(G_multi, 10)

In [ ]:
from collections import Counter
core_numbers = nx.core_number(G_multi)

hist_data = sorted(Counter(core_numbers.values()).items())

nx.info(nx.k_core(G_multi, k=89))
nx.info(nx.k_core(G_multi, k=90))


wyznacz liczbę rdzeni o największym możliwym rzędzie, o drugim możliwie największym rzędzie o trzecim możliwie największym rzędzie; jakie to są rzędy? (3);


In [ ]:
nx.info(nx.k_core(G, k=87))
nx.info(nx.k_core(G, k=88))
nx.info(nx.k_core(G, k=89))


wykreśl rozkład stopni wierzchołków (1);


In [ ]:
import matplotlib.pyplot as plt
plt.scatter(*zip(*hist_data))


wyznacz wykładnik rozkładu potęgowego metodą regresji dla dopełnienia dystrybuanty rozkładu stopni, dla przedziałów rozlokowanych logarytmicznie (3);


wyznacz wykres Hilla (3).